In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock3D(nn.Module):
    """Basic 3D residual block"""
    
    def __init__(self, in_channels, out_channels, stride=1, dropout_rate=0.0):
        super(BasicBlock3D, self).__init__()
        
        self.bn1 = nn.BatchNorm3d(in_channels)
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, 
                              stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(out_channels)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, 
                              stride=1, padding=1, bias=False)
        
        self.dropout = nn.Dropout3d(dropout_rate) if dropout_rate > 0 else None
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, 
                         stride=stride, bias=False),
                nn.BatchNorm3d(out_channels)
            )
    
    def forward(self, x):
        out = F.leaky_relu(self.bn1(x), negative_slope=0.01)
        out = self.conv1(out)
        out = F.leaky_relu(self.bn2(out), negative_slope=0.01)
        
        if self.dropout:
            out = self.dropout(out)
            
        out = self.conv2(out)
        out += self.shortcut(x)
        
        return out

class ModifiedWideResNet3D(nn.Module):
    """Modified WideResNet3D for larger input and binary classification"""
    
    def __init__(self, input_size=(64, 64, 64), width=2, num_classes=2, dropout_rate=0.3):
        super(ModifiedWideResNet3D, self).__init__()
        
        k = width
        nChannels = [16, 16*k, 32*k, 64*k, 128*k]
        
        # Initial convolution - downsample immediately for large inputs
        self.conv1 = nn.Conv3d(1, nChannels[0], kernel_size=7, stride=2, 
                              padding=3, bias=False)
        self.bn1 = nn.BatchNorm3d(nChannels[0])
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        
        # Residual blocks with progressive downsampling
        self.block1 = self._make_layer(nChannels[0], nChannels[1], 2, stride=1, dropout_rate=dropout_rate)
        self.block2 = self._make_layer(nChannels[1], nChannels[2], 2, stride=2, dropout_rate=dropout_rate)
        self.block3 = self._make_layer(nChannels[2], nChannels[3], 2, stride=2, dropout_rate=dropout_rate)
        self.block4 = self._make_layer(nChannels[3], nChannels[4], 2, stride=2, dropout_rate=dropout_rate)
        
        # Final layers
        self.adaptive_pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(nChannels[4], num_classes)
        
        self._initialize_weights()
    
    def _make_layer(self, in_channels, out_channels, num_blocks, stride, dropout_rate):
        layers = []
        layers.append(BasicBlock3D(in_channels, out_channels, stride, dropout_rate))
        
        for i in range(1, num_blocks):
            layers.append(BasicBlock3D(out_channels, out_channels, 1, dropout_rate))
            
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.leaky_relu(self.bn1(x), negative_slope=0.01)
        x = self.maxpool(x)
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        
        x = self.adaptive_pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.inplanes = 64
        
        self.conv1 = nn.Conv3d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(BasicBlock, 64, 2)
        self.layer2 = self._make_layer(BasicBlock, 128, 2, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 2, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(512, 2)  # Binary classification

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv3d(self.inplanes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(planes)
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

def resnet18():
    """3D ResNet-18 for binary classification"""
    return ResNet18()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_path = r"c:\Users\kanan\Downloads\erosion_3D\content\checkpoints\best_model.pth"
image_size = (254, 254, 254)

# Create model instance
model = ModifiedWideResNet3D(
    input_size=image_size,
    width=2,
    num_classes=2
)

# Load the saved weights
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# Set to evaluation mode
model.eval()
model.to(device)

ModifiedWideResNet3D(
  (conv1): Conv3d(1, 16, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (block1): Sequential(
    (0): BasicBlock3D(
      (bn1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (dropout): Dropout3d(p=0.3, inplace=False)
      (shortcut): Sequential(
        (0): Conv3d(16, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
        (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )


In [ ]:
import nibabel as nib
import numpy as np

def predict_single_nifti(model, nifti_path, target_size=(64, 64, 64), device=None):
    """
    Predict a single NIfTI image using the trained model
    
    Args:
        model: Trained PyTorch model
        nifti_path: Path to the .nii.gz file
        target_size: Target size to resize image to (depth, height, width)
        device: Device to run inference on
    
    Returns:
        prediction: Predicted class (0 or 1)
        confidence: Confidence score
        probabilities: Raw probabilities for both classes
    """
    
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Ensure model is in eval mode
    model.eval()
    model.to(device)
    
    # Load and preprocess the image (same as your dataset)
    image = load_and_preprocess_nifti(nifti_path, target_size)
    
    # Convert to tensor and add batch dimension
    image_tensor = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, D, H, W)
    image_tensor = image_tensor.to(device)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        predicted = torch.argmax(probabilities, dim=1)
        confidence = probabilities[0, predicted].item()
    
    return predicted.item(), confidence, probabilities[0].cpu().numpy()


def load_and_preprocess_nifti(filepath, target_size=(64, 64, 64)):
    """
    Load and preprocess a single NIfTI file
    This replicates the preprocessing logic from your dataset
    """
    
    # Load NIfTI image
    nii_img = nib.load(filepath)
    image = nii_img.get_fdata()
    
    # Handle different input dimensions
    if len(image.shape) == 4:
        # If 4D, take the first volume
        image = image[:, :, :, 0]
    
    # Normalize to [0, 1] (same as your _normalize_image method)
    image = normalize_image(image)
    
    # Resize if needed (same as your _resize_image method)
    if image.shape != target_size:
        image = resize_image(image, target_size)
    
    return image


def normalize_image(image):
    """Normalize image to [0, 1] range - copied from your dataset"""
    # Remove NaN and infinity values
    image = np.nan_to_num(image, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Normalize to [0, 1]
    min_val = np.min(image)
    max_val = np.max(image)
    if max_val > min_val:
        image = (image - min_val) / (max_val - min_val)
    else:
        image = np.zeros_like(image)
    
    return image


def resize_image(image, target_size):
    """Resize image using trilinear interpolation - copied from your dataset"""
    # Convert to tensor for interpolation
    image_tensor = torch.from_numpy(image).float().unsqueeze(0).unsqueeze(0)
    
    # Resize using trilinear interpolation
    resized = F.interpolate(
        image_tensor,
        size=target_size,
        mode='trilinear',
        align_corners=False
    )
    
    return resized.squeeze(0).squeeze(0).numpy()

In [4]:
image_path = r'd:\Kananat\Data\training_dataset_3D\training_dataset_erosion\test\0\57-36050 L_adjustedBG.nii.gz'

prediction, confidence, probabilities = predict_single_nifti(
    model, 
    image_path, 
    target_size=image_size  # Use the same target_size as training
)

print(f"Predicted class: {prediction}, Confidence: {confidence:.4f}, Probabilities: {probabilities}")

Predicted class: 0, Confidence: 0.9060, Probabilities: [0.9059875  0.09401254]


In [13]:
import os
import csv

data_dir = r'D:\Kananat\Data\training_dataset_3D\training_dataset_erosion\val'

result_from_images = []
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.nii') or file.endswith('.nii.gz'):
            real_label = int(root.split(os.sep)[-1])  # Get the real label from the folder name
            file_path = os.path.join(root, file)
            patient_id = file.split('_')[0]
            
            predict_label, confidence, probabilities = predict_single_nifti(
                model, 
                file_path, 
                target_size=image_size  # Use the same target_size as training
            )
            print(f"File: {file_path}, Predicted class: {predict_label}, Confidence: {confidence:.4f}, Probabilities: {probabilities}")

            print(patient_id, real_label, predict_label, confidence)
            result_from_images.append((patient_id, real_label, predict_label, confidence))

save_path = rf"{data_dir}\predictions.txt"
with open(save_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write data
    for patient_id ,real, predicted_class, conf in result_from_images:
        writer.writerow([patient_id ,real, predicted_class, conf])

File: D:\Kananat\Data\training_dataset_3D\training_dataset_erosion\val\0\49-18165 R_adjustedBG.nii.gz, Predicted class: 0, Confidence: 0.5304, Probabilities: [0.53043586 0.46956405]
49-18165 R 0 0 0.5304358601570129
File: D:\Kananat\Data\training_dataset_3D\training_dataset_erosion\val\0\50-30909 R_adjustedBG.nii.gz, Predicted class: 0, Confidence: 0.5039, Probabilities: [0.5039019  0.49609813]
50-30909 R 0 0 0.5039018988609314
File: D:\Kananat\Data\training_dataset_3D\training_dataset_erosion\val\0\56-18364 L_adjustedBG.nii.gz, Predicted class: 1, Confidence: 0.9214, Probabilities: [0.07861178 0.92138827]
56-18364 L 0 1 0.9213882684707642
File: D:\Kananat\Data\training_dataset_3D\training_dataset_erosion\val\0\58-42016 L_adjustedBG.nii.gz, Predicted class: 1, Confidence: 0.8477, Probabilities: [0.15227446 0.8477256 ]
58-42016 L 0 1 0.8477255702018738
File: D:\Kananat\Data\training_dataset_3D\training_dataset_erosion\val\0\58-42016 R_adjustedBG.nii.gz, Predicted class: 1, Confidence: 0

In [18]:
csv_file = r"d:\Kananat\Data\training_dataset_3D\training_dataset_erosion\test\predictions.txt"

with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    data = list(csv_reader)

print(len(data))

39


In [20]:
correct = sum(1 for row in data if int(row[1]) == int(row[2]))
print(correct)

27


In [21]:
print(27/39)

0.6923076923076923
